In [1]:
import torch
from torchinfo import summary
from model import model_builder
from remapping_rules import remap_rgb_branch_15, remap_depth_branch_15

/home/gkmo/workspace/rgbd-yolov2/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
fusion_model = model_builder(num_classes=3, model_type="rgbd", fuse_layer=15)

All RGB weights are equivalent.

All Depth weights are equivalent.


In [2]:
# model = ln.models.Darknet19(1000)
# model.load('weights/darknet19_448.weights')

# # Save as PyTorch weight file (Not strictly necessary, but it is faster than darknet weight files)
# model.save('weights/darknet19_448.pt')

# # Converting Darknet19 weights to Yolo (This is the same as the darknet19_448.conv.23.weights from darknet)
# model.save('weights/yolo-pretrained_darknet.pt', remap=ln.models.YoloV2.remap_darknet19)

# # Load yolo weights (Requires `strict=False`, because not all layers have weights in this file)
# detection_model = ln.models.YoloV2(3)
# detection_model.load('weights/yolo-pretrained_darknet.pt', strict=False)

## Test the weights of rgbd network

In [2]:
rgb_model = model_builder(num_classes=3, model_type="rgb")

/home/gkmo/workspace/rgbd-yolov2/.venv/lib/python3.10/site-packages/lightnet/network/module/_lightnet.py:293: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.loa

In [13]:
for i in rgb_model.named_parameters():
    print(i[0], i[1].size())

layers.0.1_convbatch_regular.layers.0.weight torch.Size([32, 3, 3, 3])
layers.0.1_convbatch_regular.layers.1.weight torch.Size([32])
layers.0.1_convbatch_regular.layers.1.bias torch.Size([32])
layers.0.1_convbatch_fusion.layers.0.weight torch.Size([32, 1, 3, 3])
layers.0.1_convbatch_fusion.layers.1.weight torch.Size([32])
layers.0.1_convbatch_fusion.layers.1.bias torch.Size([32])
layers.1.regular.3_convbatch.layers.0.weight torch.Size([64, 32, 3, 3])
layers.1.regular.3_convbatch.layers.1.weight torch.Size([64])
layers.1.regular.3_convbatch.layers.1.bias torch.Size([64])
layers.1.regular.5_convbatch.layers.0.weight torch.Size([128, 64, 3, 3])
layers.1.regular.5_convbatch.layers.1.weight torch.Size([128])
layers.1.regular.5_convbatch.layers.1.bias torch.Size([128])
layers.1.regular.6_convbatch.layers.0.weight torch.Size([64, 128, 1, 1])
layers.1.regular.6_convbatch.layers.1.weight torch.Size([64])
layers.1.regular.6_convbatch.layers.1.bias torch.Size([64])
layers.1.regular.7_convbatch.la

In [ ]:
summary(model=rgb_model, 
        input_size=(32, 3, 416, 416), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                       Input Shape          Output Shape         Param #              Trainable
YoloV2 (YoloV2)                               [32, 1, 416, 416]    [32, 40, 13, 13]     --                   True
├─FeatureExtractor (backbone)                 [32, 1, 416, 416]    [32, 1024, 13, 13]   --                   True
│    └─Sequential (module)                    [32, 1, 416, 416]    [32, 1024, 13, 13]   --                   True
│    │    └─Conv2dBatchAct (1_convbatch)      [32, 1, 416, 416]    [32, 32, 416, 416]   352                  True
│    │    └─MaxPool2d (2_max)                 [32, 32, 416, 416]   [32, 32, 208, 208]   --                   --
│    │    └─Conv2dBatchAct (3_convbatch)      [32, 32, 208, 208]   [32, 64, 208, 208]   18,560               True
│    │    └─MaxPool2d (4_max)                 [32, 64, 208, 208]   [32, 64, 104, 104]   --                   --
│    │    └─Conv2dBatchAct (5_convbatch)      [32, 64, 104, 104]   [32, 128, 104, 104] 

In [11]:
depth_model = model_builder(num_classes=3, model_type="depth")

/home/gkmo/workspace/rgbd-yolov2/.venv/lib/python3.10/site-packages/lightnet/network/module/_lightnet.py:293: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.loa

Replacing first layer with new conv: Conv2dBatchAct(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), LeakyReLU(negative_slope=0.1, inplace=True))


In [4]:
depth_pretrained = torch.load("models/depth_99.pth")

/tmp/ipykernel_25676/3341212066.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  depth_pretrained = torch.load("models/depth_99.pth")


In [6]:
type(depth_pretrained)

collections.OrderedDict

In [3]:
fusion_model = model_builder(num_classes=3, model_type="rgbd", fuse_layer=15)

AttributeError: 'collections.OrderedDict' object has no attribute 'state_dict'

In [ ]:
def check_weights(rgbd_model, rgb_model, depth_model, rgb_remap, depth_remap):
    """
    Function to check if the weights of the RGBD model are equivalent to the sum of the RGB and Depth models.
    """
    for rgb_param_name, rgbd_param_name in rgb_remap.items():
        rgb_param = rgb_model.state_dict()[rgb_param_name]
        rgbd_param = rgbd_model.state_dict()[rgbd_param_name]
        if torch.equal(rgb_param, rgbd_param):
            print(f"RGB Weights are equivalent. \n{rgb_param_name}\n{rgbd_param_name}\n")
            last_check = True
        else:
            print(f"Weights are NOT equivalent.\n{rgb_param_name}\n{rgbd_param_name}\n")
            last_check = False
            
    if last_check:
        print("All RGB weights are equivalent.\n")        

    for depth_param_name, rgbd_param_name in depth_remap.items():
        depth_param = depth_model.state_dict()[depth_param_name]
        rgbd_param = rgbd_model.state_dict()[rgbd_param_name]
        if torch.equal(depth_param, rgbd_param):
            #print(f"Depth Weights are equivalent. \n{depth_param_name}\n{rgbd_param_name}\n")
            last_check = True
        else:
            #print(f"Depth Weights are NOT equivalent.\n{depth_param_name}\n{rgbd_param_name}\n")
            last_check = False
            

    if last_check:
        print("All Depth weights are equivalent.") 


    if last_check:
        print("All weights are equivalent.")
    else:
        print("Some weights are NOT equivalent.")

In [19]:
check_weights(fusion_model, rgb_model, depth_model, remap_rgb_branch_15, remap_depth_branch_15)

RGB Weights are equivalent. 
backbone.module.1_convbatch.layers.0.weight
layers.0.1_convbatch_regular.layers.0.weight

RGB Weights are equivalent. 
backbone.module.1_convbatch.layers.1.weight
layers.0.1_convbatch_regular.layers.1.weight

RGB Weights are equivalent. 
backbone.module.1_convbatch.layers.1.bias
layers.0.1_convbatch_regular.layers.1.bias

RGB Weights are equivalent. 
backbone.module.1_convbatch.layers.1.running_mean
layers.0.1_convbatch_regular.layers.1.running_mean

RGB Weights are equivalent. 
backbone.module.1_convbatch.layers.1.running_var
layers.0.1_convbatch_regular.layers.1.running_var

RGB Weights are equivalent. 
backbone.module.1_convbatch.layers.1.num_batches_tracked
layers.0.1_convbatch_regular.layers.1.num_batches_tracked

RGB Weights are equivalent. 
backbone.module.3_convbatch.layers.0.weight
layers.1.regular.3_convbatch.layers.0.weight

RGB Weights are equivalent. 
backbone.module.3_convbatch.layers.1.weight
layers.1.regular.3_convbatch.layers.1.weight

RGB 

In [5]:
input_tensor = torch.randn(32, 3, 416, 416)
output_tensor = rgb_model(input_tensor)
print(output_tensor.shape)

torch.Size([32, 40, 13, 13])


In [17]:
rgb_model.state_dict()["backbone.module.1_convbatch.layers.1.num_batches_tracked"]

tensor(1)

In [24]:
#fusion_model.state_dict().keys()

for key, value in rgb_model.state_dict().items():
    print(key)

backbone.module.1_convbatch.layers.0.weight
backbone.module.1_convbatch.layers.1.weight
backbone.module.1_convbatch.layers.1.bias
backbone.module.1_convbatch.layers.1.running_mean
backbone.module.1_convbatch.layers.1.running_var
backbone.module.1_convbatch.layers.1.num_batches_tracked
backbone.module.3_convbatch.layers.0.weight
backbone.module.3_convbatch.layers.1.weight
backbone.module.3_convbatch.layers.1.bias
backbone.module.3_convbatch.layers.1.running_mean
backbone.module.3_convbatch.layers.1.running_var
backbone.module.3_convbatch.layers.1.num_batches_tracked
backbone.module.5_convbatch.layers.0.weight
backbone.module.5_convbatch.layers.1.weight
backbone.module.5_convbatch.layers.1.bias
backbone.module.5_convbatch.layers.1.running_mean
backbone.module.5_convbatch.layers.1.running_var
backbone.module.5_convbatch.layers.1.num_batches_tracked
backbone.module.6_convbatch.layers.0.weight
backbone.module.6_convbatch.layers.1.weight
backbone.module.6_convbatch.layers.1.bias
backbone.mod

In [22]:
rgbd_regular_weight = fusion_model.state_dict()['layers.1.regular.11_convbatch.layers.1.weight'].cpu()
rgb_weight = rgb_model.state_dict()['backbone.module.11_convbatch.layers.1.weight'].cpu()

In [30]:
depth_weight = depth_pretrained['backbone.module.10_convbatch.layers.1.weight'].cpu()
rgbd_fusion_fusion_weight = fusion_model.state_dict()['layers.1.fusion.10_convbatch.layers.1.weight'].cpu()

In [31]:
torch.equal(depth_weight, rgbd_fusion_fusion_weight)

True

In [32]:
torch.equal(rgbd_regular_weight, rgb_weight)

True